# Predicting House Sale Prices

## By using linear regression's models

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [2]:
AmesHousing= pd.read_csv("AmesHousing.tsv", delimiter="\t")

In [3]:
def transform_features(df):
    return df

def select_features(df):
    return df[["Gr Liv Area","SalePrice"]]

def train_and_test(df):
    train= df[:1460]
    test= df[1460:]
    
    ### Let's Train a model with just numerical columns
    
    numeric_train= train.select_dtypes(include=["integer","float"])
    features= numeric_train.columns.drop("SalePrice")
    
    ### let's create linear regression's model
    
    lr= LinearRegression()
    lr.fit(train[features],train["SalePrice"])
    predictions= lr.predict(test[features])
    rmse= mean_squared_error(test["SalePrice"],predictions)**(1/2)
    return rmse

In [4]:
### Let's test this model

df= transform_features(AmesHousing)
df= select_features(df)
print(train_and_test(df))

57088.25161263909


## Let's modify the transform_features function

In [5]:
### Let's interest on the features with less 5% of the missing values
null_features= AmesHousing.isnull().sum()

In [6]:
# Filter Series to columns containing >5% missing values
drop_missing_cols = null_features[(null_features > len(df)/20)].sort_values()

# Drop those columns from the data frame. Note the use of the .index accessor
AmesHousing = AmesHousing.drop(drop_missing_cols.index, axis=1)

In [7]:
### Series object: column name -> number of missing values
text_mv_counts = AmesHousing.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

## Filter Series to columns containing *any* missing values
drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]

AmesHousing = AmesHousing.drop(drop_missing_cols_2.index, axis=1)

In [8]:
## Compute column-wise missing value counts
num_missing = AmesHousing.select_dtypes(include=['int', 'float']).isnull().sum()
fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
fixable_numeric_cols

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

In [9]:
### Compute the most common value for each column in `fixable_nmeric_missing_cols`.
replacement_values_dict = AmesHousing[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
replacement_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [10]:
### Use `pd.DataFrame.fillna()` to replace missing values.
AmesHousing = AmesHousing.fillna(replacement_values_dict)

In [11]:
## Verify that every column has 0 missing values
AmesHousing.isnull().sum().value_counts()

0    64
dtype: int64

What new features can we create, that better capture the information in some of the features?

In [12]:
years_sold = AmesHousing['Yr Sold'] - AmesHousing['Year Built']
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [13]:
years_since_remod = AmesHousing['Yr Sold'] - AmesHousing['Year Remod/Add']
years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [14]:
## Create new columns
AmesHousing['Years Before Sale'] = years_sold
AmesHousing['Years Since Remod'] = years_since_remod

## Drop rows with negative values for both of these new features
AmesHousing = AmesHousing.drop([1702, 2180, 2181], axis=0)

## No longer need original year columns
AmesHousing = AmesHousing.drop(["Year Built", "Year Remod/Add"], axis = 1)

In [15]:
## Drop columns that aren't useful for ML
AmesHousing = AmesHousing.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
AmesHousing = AmesHousing.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

In [21]:
def transform_features(df):
    ### deleting of the 5% or more of the columns with missing values
    features_null= df.isnull().sum()
    drop_missing_col= features_null[features_null>(len(df)/20)].sort_values()
    df= df.drop(drop_missing_col.index,axis=1)
    
    ### deleting of the missing values of the text columns
    text_mv_counts= df.select_dtypes(include="object").isnull().sum().sort_values(ascending=False)
    drop_missing_col_2= text_mv_counts[text_mv_counts>0]
    df= df.drop(drop_missing_col_2.index,axis=1)
    
    ### selecting of the missing values of the numerical columns with missing values
    num_missing= df.select_dtypes(include=["integer","float"]).isnull().sum()  
    fixable_num_col= num_missing[(num_missing<(len(df)/20)) & (num_missing>0)]
    
    ### Let's replace that's missing values by most common value for each column
    replacement_values_dict= df[fixable_num_col.index].mode().to_dict(orient="record")[0]
    df= df.fillna(replacement_values_dict)
    
    ### creating of the new columns
    df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
    df['Years Since Remod'] = df['Yr Sold'] - df['Year Remod/Add']
    
    ## Drop rows with negative values for both of these new features
    df = df.drop([1702, 2180, 2181], axis=0)

    ## No longer need original year columns
    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
    
    ## Drop columns that aren't useful for ML
    df = df.drop(["PID", "Order"], axis=1)

    ## Drop columns that leak info about the final sale
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    return df

def select_features(df):
    return df[["Gr Liv Area","SalePrice"]]

def train_and_test(df):
    train= df[:1460]
    test= df[1460:]
    
    ### Let's Train a model with just numerical columns
    
    numeric_train= train.select_dtypes(include=["integer","float"])
    features= numeric_train.columns.drop("SalePrice")
    
    ### let's create linear regression's model
    
    lr= LinearRegression()
    lr.fit(train[features],train["SalePrice"])
    predictions= lr.predict(test[features])
    rmse= mean_squared_error(test["SalePrice"],predictions)**(1/2)
    return rmse

In [24]:
### Let's test this model

AmesHousing= pd.read_csv("AmesHousing.tsv", delimiter="\t")
df= transform_features(AmesHousing)
df= select_features(df)
print(train_and_test(df))

55275.36731241307


## Now, let's modify the select features function

In [26]:
transform_f= transform_features(AmesHousing)
numerical_features= transform_f.select_dtypes(include=["float","integer"])
numerical_features.head()

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years Before Sale,Years Since Remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,212,34,0,0,0,0,0,189900,13,12


In [29]:
numerical_features_corr= numerical_features.corr()["SalePrice"].abs().sort_values()
numerical_features_corr

BsmtFin SF 2         0.006127
Misc Val             0.019273
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: Sale

In [36]:
### Let's just keep clumns with correlation's coefficient superior to 0.4
numerical_features = numerical_features.drop(numerical_features_corr[numerical_features_corr < 0.4].index, axis=1)
numerical_features

,Overall Qual,Mas Vnr Area,BsmtFin SF 1,Total Bsmt SF,1st Flr SF,Gr Liv Area,Full Bath,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,SalePrice,Years Before Sale,Years Since Remod
0,6,112.0,639.0,1080.0,1656,1656,1,7,2,2.0,528.0,215000,50,50
1,5,0.0,468.0,882.0,896,896,1,5,0,1.0,730.0,105000,49,49
2,6,108.0,923.0,1329.0,1329,1329,1,6,0,1.0,312.0,172000,52,52
3,7,0.0,1065.0,2110.0,2110,2110,2,8,2,2.0,522.0,244000,42,42
4,5,0.0,791.0,928.0,928,1629,2,6,1,2.0,482.0,189900,13,12
5,6,20.0,602.0,926.0,926,1604,2,7,1,2.0,470.0,195500,12,12
6,8,0.0,616.0,1338.0,1338,1338,2,6,0,2.0,582.0,213500,9,9
7,8,0.0,263.0,1280.0,1280,1280,2,5,0,2.0,506.0,191500,18,18
8,8,0.0,1180.0,1595.0,1616,1616,2,5,1,2.0,608.0,236500,15,14
9,7,0.0,0.0,994.0,1028,1804,2,7,1,2.0,442.0,189000,11,11


In [37]:
### Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [38]:
transform_cat_cols = []
for col in nominal_features:
    if col in transform_f.columns:
        transform_cat_cols.append(col)
## How many unique values in each categorical column?
uniqueness_counts = transform_f[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
## Aribtrary cutoff of 10 unique values (worth experimenting)
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_f = transform_f.drop(drop_nonuniq_cols, axis=1)

In [39]:
## Select just the remaining text columns and convert to categorical
text_cols = transform_f.select_dtypes(include=['object'])
for col in text_cols:
    transform_f[col] = transform_f[col].astype('category')
    
## Create dummy columns and add back to the dataframe!
transform_f = pd.concat([
    transform_f, 
    pd.get_dummies(transform_f.select_dtypes(include=['category']))
], axis=1).drop(text_cols,axis=1)

##### Let's update these pipeline functions

In [45]:
def transform_features(df):
    ### deleting of the 5% or more of the columns with missing values
    features_null= df.isnull().sum()
    drop_missing_col= features_null[features_null>(len(df)/20)].sort_values()
    df= df.drop(drop_missing_col.index,axis=1)
    
    ### deleting of the missing values of the text columns
    text_mv_counts= df.select_dtypes(include="object").isnull().sum().sort_values(ascending=False)
    drop_missing_col_2= text_mv_counts[text_mv_counts>0]
    df= df.drop(drop_missing_col_2.index,axis=1)
    
    ### selecting of the missing values of the numerical columns with missing values
    num_missing= df.select_dtypes(include=["integer","float"]).isnull().sum()  
    fixable_num_col= num_missing[(num_missing<(len(df)/20)) & (num_missing>0)]
    
    ### Let's replace that's missing values by most common value for each column
    replacement_values_dict= df[fixable_num_col.index].mode().to_dict(orient="record")[0]
    df= df.fillna(replacement_values_dict)
    
    ### creating of the new columns
    df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
    df['Years Since Remod'] = df['Yr Sold'] - df['Year Remod/Add']
    
    ## Drop rows with negative values for both of these new features
    df = df.drop([1702, 2180, 2181], axis=0)

    ## No longer need original year columns
    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
    
    ## Drop columns that aren't useful for ML
    df = df.drop(["PID", "Order"], axis=1)

    ## Drop columns that leak info about the final sale
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    return df

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    return df

def train_and_test(df, k=0):
    ### Let's create a model with just numerical columns
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = LinearRegression()
    
    ### cross validation on different k's values
    if k==0:
        train= df[:1460]
        test= df[1460:]
        lr.fit(train[features],train["SalePrice"])
        predictions= lr.predict(test[features])
        rmse= mean_squared_error(test["SalePrice"],predictions)**(1/2)
        return rmse
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

In [46]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

[27287.63546433653, 26339.28363185429, 36487.90155939892, 27534.165816260156]


29412.24661796247